In [ ]:
#import libraries
import os
import requests
import pandas as pd
from flatten_json import flatten

In [ ]:
#check contents of directory
os.listdir()

In [ ]:
#An access token must be included in all requests as either a URL parameter (access_token) or HTTP request header (Authorization)

#GET /api/deposit/depositions?access_token=<ACCESS_TOKEN>

#GET /api/deposit/depositions
#Authorization: Bearer <ACCESS_TOKEN>

In [ ]:
#import pesonal access token, stored in Zenodo_access_token.py
!python 'Zenodo_access_token.py'
#not sure why this isn't working?? the file is definitely there

In [ ]:
#this works in the meantime
exec(open("Zenodo_access_token.py").read())

In [ ]:
#check that loaded ACCESS_TOKEN variable
%whos

In [ ]:
#General Zenodo requests format
#response = requests.get('https://zenodo.org/api/records',
#                        params={'q': 'my title',
#                                'access_token': ACCESS_TOKEN})
#print(response.json())

In [ ]:
#example search
response = requests.get('https://zenodo.org/api/records',
                        params={'q': 'machine learning',
                                'access_token': ACCESS_TOKEN})
#print(response.json())

In [ ]:
#EXPLORATORY NOTES

#exploring with Postman - GET https://zenodo.org/api/records/?sort=bestmatch&q=machine+learning&page=2&size=10
#looks like results are paginated, so need to get total number of pages, then go through
#can have size up to max of 100 (100 results per page looks like)

#first section of json is summary of results (number by file type, format, etc)
#"aggregations" section
#this is good info - want to extract as separate summary table

#the section with details is "hits"
#we want to extract and collapse to have hits metadata as tabular form

#can extract downloads - downstream, could filter to top 200 objects by most downloaded 
#(for those that aren't just papers, but actually include other components)

#will need to filter by access rights
#do we want everything, or just "open"?
#do we just want most recent version? (probably)

In [ ]:
#COLLAPSE METHOD 1

#collapse json using flatten_json (https://github.com/amirziai/flatten)
zenodo_search = flatten(response.json())
print(zenodo_search)

#this is a FULL collapse - good start but not quite what we want

In [ ]:
type(zenodo_search)

In [ ]:
#COLLAPSE METHOD 2

#alternate flattening pandas method
zenodo_search_pd = pd.io.json.json_normalize(response.json())
zenodo_search_pd

#potential for splitting out aggregation vs hits
#could split into two dfs and then continue to collapse to columns

In [ ]:
#COLLAPSE METHOD 3

#We want something like this (maybe?): https://github.com/amirziai/flatten

dic = [
    {"a": 1, "b": 2, "c": {"d": 3, "e": 4}},
    {"a": 0.5, "c": {"d": 3.2}},
    {"a": 0.8, "b": 1.8},
]

dic_flattened = [flatten(d) for d in dic]

df = pd.DataFrame(dic_flattened)

df